# DataFrames.jl 介紹 (二): Joins 與 Split-Apply-Combine Strategy

## Day 018 作業1：The Split-Apply-Combine Strategy

同 Day 017，請載入 2020/4/2 COVID-19 Daily Report 資料集，計算每個國家/地區的累計確診人數，並列出所有資料筆數。

In [1]:
using DataFrames, CSV

In [5]:
df = CSV.read("04-02-2020.csv")
show(df[1,:],allcols = true)

DataFrameRow
│ Row │ FIPS   │ Admin2    │ Province_State │ Country_Region │
│     │ Int64⍰ │ String⍰   │ String⍰        │ String         │
├─────┼────────┼───────────┼────────────────┼────────────────┤
│ 1   │ 45001  │ Abbeville │ South Carolina │ US             │

│ Row │ Last_Update         │ Lat      │ Long_    │ Confirmed │ Deaths │
│     │ String              │ Float64⍰ │ Float64⍰ │ Int64     │ Int64  │
├─────┼─────────────────────┼──────────┼──────────┼───────────┼────────┤
│ 1   │ 2020-04-02 23:25:27 │ 34.2233  │ -82.4617 │ 6         │ 0      │

│ Row │ Recovered │ Active │ Combined_Key                  │
│     │ Int64     │ Int64  │ String                        │
├─────┼───────────┼────────┼───────────────────────────────┤
│ 1   │ 0         │ 0      │ Abbeville, South Carolina, US │

計算計算每個國家/地區的累計確診人數

【提示】可使用 groupby 相關的函式

In [31]:
df2 = by(df,:Country_Region, totalConfirmed = :Confirmed => sum)
show(df2,allrows=true)

181×2 DataFrame
│ Row │ Country_Region                   │ totalConfirmed │
│     │ String                           │ Int64          │
├─────┼──────────────────────────────────┼────────────────┤
│ 1   │ US                               │ 243453         │
│ 2   │ Canada                           │ 11284          │
│ 3   │ United Kingdom                   │ 34173          │
│ 4   │ China                            │ 82432          │
│ 5   │ Netherlands                      │ 14788          │
│ 6   │ Australia                        │ 5116           │
│ 7   │ Denmark                          │ 3573           │
│ 8   │ France                           │ 59929          │
│ 9   │ Afghanistan                      │ 273            │
│ 10  │ Albania                          │ 277            │
│ 11  │ Algeria                          │ 986            │
│ 12  │ Andorra                          │ 428            │
│ 13  │ Angola                           │ 8              │
│ 14  │ Antigua and Barb

│ 134 │ Paraguay                         │ 77             │
│ 135 │ Peru                             │ 1414           │
│ 136 │ Philippines                      │ 2633           │
│ 137 │ Poland                           │ 2946           │
│ 138 │ Portugal                         │ 9034           │
│ 139 │ Qatar                            │ 949            │
│ 140 │ Romania                          │ 2738           │
│ 141 │ Russia                           │ 3548           │
│ 142 │ Rwanda                           │ 84             │
│ 143 │ Saint Kitts and Nevis            │ 9              │
│ 144 │ Saint Lucia                      │ 13             │
│ 145 │ Saint Vincent and the Grenadines │ 2              │
│ 146 │ San Marino                       │ 245            │
│ 147 │ Saudi Arabia                     │ 1885           │
│ 148 │ Senegal                          │ 195            │
│ 149 │ Serbia                           │ 1171           │
│ 150 │ Seychelles                      

## 作業2：請列出美國各區域或事件 (依 Province_State 欄) 的累積確診、死亡、康復人數。

【提示】使用 Split-Apply-Combine Strategy 時，有幾種不同的函式可以使用，請參照 `DataFrames.jl` 官方文件 [https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.groupby](https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.groupby)

In [33]:
df_US = filter(x -> x.Country_Region == "US", df);
gp = groupby(df_US,:Province_State);
println("Total $(size(df_US,1)) rows are splitted into $(length(gp)) groups")  # number of groups
US_summary = by(df_US, :Province_State; totalComfirmed = :Confirmed => sum, totalDeaths = :Deaths => sum, recovered = :Recovered => sum);
show(US_summary,allrows = true)


Total 2312 rows are splitted into 58 groups
58×4 DataFrame
│ Row │ Province_State           │ totalComfirmed │ totalDeaths │ recovered │
│     │ Union{Missing, String}   │ Int64          │ Int64       │ Int64     │
├─────┼──────────────────────────┼────────────────┼─────────────┼───────────┤
│ 1   │ South Carolina           │ 1554           │ 31          │ 0         │
│ 2   │ Louisiana                │ 9159           │ 310         │ 0         │
│ 3   │ Virginia                 │ 1706           │ 41          │ 0         │
│ 4   │ Idaho                    │ 776            │ 9           │ 0         │
│ 5   │ Iowa                     │ 614            │ 11          │ 0         │
│ 6   │ Missouri                 │ 1857           │ 22          │ 0         │
│ 7   │ Oklahoma                 │ 881            │ 34          │ 0         │
│ 8   │ Colorado                 │ 3342           │ 80          │ 0         │
│ 9   │ Illinois                 │ 7695           │ 163         │ 0         │
│ 10 

In [35]:
# alternatively (from the reference answer)
df_US_grouped = groupby(df[df.Country_Region .== "US", :], :Province_State)
show(combine(df_US_grouped, :Confirmed => sum, :Deaths => sum, :Recovered => sum), allrows=true)

58×4 DataFrame
│ Row │ Province_State           │ Confirmed_sum │ Deaths_sum │ Recovered_sum │
│     │ Union{Missing, String}   │ Int64         │ Int64      │ Int64         │
├─────┼──────────────────────────┼───────────────┼────────────┼───────────────┤
│ 1   │ South Carolina           │ 1554          │ 31         │ 0             │
│ 2   │ Louisiana                │ 9159          │ 310        │ 0             │
│ 3   │ Virginia                 │ 1706          │ 41         │ 0             │
│ 4   │ Idaho                    │ 776           │ 9          │ 0             │
│ 5   │ Iowa                     │ 614           │ 11         │ 0             │
│ 6   │ Missouri                 │ 1857          │ 22         │ 0             │
│ 7   │ Oklahoma                 │ 881           │ 34         │ 0             │
│ 8   │ Colorado                 │ 3342          │ 80         │ 0             │
│ 9   │ Illinois                 │ 7695          │ 163        │ 0             │
│ 10  │ Indiana          